In [125]:
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from gensim import corpora, models, similarities
import warnings
import nltk
from nltk.corpus import stopwords
from pprint import pprint
import string

warnings.filterwarnings("ignore")

In [126]:
FOLDER_PATH = './data/bbc/tech'

stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

In [127]:
#функция для создания списка текстов
def create_text_list(folder_path = FOLDER_PATH):
    file_list = sorted(os.listdir(folder_path))
    text_list = []
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as file:
            text = file.read()
            text_list.append(text)
    return text_list



In [128]:
text_list = create_text_list()

In [129]:
def delete_stop_words(list):
    return [[word for word in document.lower().split() if word not in stop_words] for document in text_list]

In [130]:
texts = delete_stop_words(text_list)

In [131]:
#создадим словарь и преобразуем данные, используя TF-IDF модель
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

**Протестируем модель**

In [136]:
def testing(query):
    query_bow = dictionary.doc2bow(query.lower().split())
    query_tfidf = tfidf[query_bow]
    index = similarities.MatrixSimilarity(corpus_tfidf, num_features=len(dictionary))
    similarities_score = index[query_tfidf]
    similarities_score = sorted(enumerate(similarities_score), key=lambda item: -item[1])
    print('Query:', query, '\n')

    for i in similarities_score[:3]:
        print('Text №', i[0]+1, 'similarities =', i[1], '\n')
        print(text_list[i[0]])
        print('-'*50)

In [137]:
testing('rfid tags')

Query: rfid tags 

Text № 61 similarities = 0.7372147 

Concern over RFID tags

Consumers are very concerned about the use of radio frequency ID (RFID) tags in shops, a survey says.

More than half of 2,000 people surveyed said they had privacy worries about the tags, which can be used to monitor stock on shelves or in warehouses. Some consumer groups have expressed concern that the tags could be used to monitor shoppers once they had left shops with their purchases. The survey showed that awareness of tags among consumers in Europe was low. The survey of consumers in the UK, France, Germany and the Netherlands was carried out by consultancy group Capgemini. The firm works on behalf of more than 30 firms who are seeking to promote the growth of RFID technology. The tags are a combination of computer chip and antenna which can be read by a scanner - each item contains a unique identification number.

More than half (55%) of the respondents said they were either concerned or very concern

In [138]:
testing('google chrome')

Query: google chrome 

Text № 186 similarities = 0.38802734 

China 'blocks Google news site'

China has been accused of blocking access to Google News by the media watchdog, Reporters Without Borders.

The Paris-based pressure group said the English-language news site had been unavailable for the past 10 days. It said the aim was to force people to use a Chinese edition of the site which, according to the watchdog, does not include critical reports. Google told the BBC News website it was aware of the problems and was investigating the causes.

China is believed to extend greater censorship over the net than any other country in the world.

A net police force monitors websites and e-mails, and controls on gateways connecting the country to the global internet are designed to prevent access to critical information. Popular Chinese portals such as Sina.com and Sohu.com maintain a close eye on content and delete politically sensitive comments. And all 110,000 net cafes in the country hav

In [139]:
testing('microsoft')

Query: microsoft 

Text № 3 similarities = 0.2936793 

Microsoft seeking spyware trojan

Microsoft is investigating a trojan program that attempts to switch off the firm's anti-spyware software.

The spyware tool was only released by Microsoft in the last few weeks and has been downloaded by six million people. Stephen Toulouse, a security manager at Microsoft, said the malicious program was called Bankash-A Trojan and was being sent as an e-mail attachment. Microsoft said it did not believe the program was widespread and recommended users to use an anti-virus program. The program attempts to disable or delete Microsoft's anti-spyware tool and suppress warning messages given to users.

It may also try to steal online banking passwords or other personal information by tracking users' keystrokes.

Microsoft said in a statement it is investigating what it called a criminal attack on its software. Earlier this week, Microsoft said it would buy anti-virus software maker Sybari Software to i

Вывод: Перед нами стояла задача поиска текста по запросу. Для решения задачи было использовано tf-idf-преобразование. На первом тестовом запросе в качестве результата модель выдала тексты очень близкие к запросу (метрика схожести 0,73 для первых двух текстов).